In [ ]:
import torch, os
from diffusers import StableDiffusionXLPipeline
from tqdm import tqdm
from datetime import datetime
import papermill as pm

# -------- SETTINGS ---------
# MODEL_ID = "runwayml/stable-diffusion-v1-5"
MODEL_ID = "stabilityai/stable-diffusion-xl-beta-v2-2"
# -------- PARAMETERS (can be overridden when running via papermill) ---------
PROMPTS = ['A photorealistic photograph of a restaurant scene: a single customer ordering food, a waiter presenting the menu, natural lighting, sharp focus, realistic textures, clean background, 16:9 aspect ratio, 4K quality.']
NEGATIVE_PROMPT = "blurry, low quality, distorted, watermark, duplicate, multiple identical people, clones, repetition"
OUTPUT_DIR = "generated_images"
IMG_SIZE = (1080, 1920)  # HD ~16:9
GUIDANCE = 7.5
STEPS = 25
SEED = 42
USE_GPU = True
# ----------------------------

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Detect device
device = "cuda" if USE_GPU and torch.cuda.is_available() else "cpu"
print("GPU available:", torch.cuda.get_device_name(0) if device=="cuda" else "Running on CPU.")

# Load pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
).to(device)

generator = torch.manual_seed(SEED)

# Generate images (1 per prompt)
for prompt in tqdm(PROMPTS, desc="Generating"):
    image = pipe(
        prompt,
        negative_prompt=NEGATIVE_PROMPT,
        guidance_scale=GUIDANCE,
        num_inference_steps=STEPS,
        generator=generator,
        height=IMG_SIZE[0],
        width=IMG_SIZE[1],
    ).images[0]

    # Save with timestamp
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"img_{ts}.png"
    save_path = os.path.join(OUTPUT_DIR, filename)
    image.save(save_path)
    print(f"Saved image: {save_path}")

# Move to Kaggle Output folder if running on Kaggle
if "KAGGLE_URL_BASE" in os.environ:
    kaggle_output = "/kaggle/working/output"
    os.makedirs(kaggle_output, exist_ok=True)
    for file in os.listdir(OUTPUT_DIR):
        src = os.path.join(OUTPUT_DIR, file)
        dst = os.path.join(kaggle_output, file)
        os.rename(src, dst)
    print(f"Images moved to Kaggle Output folder: {kaggle_output}")